#Set-Up

In [1]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

     |████████████████████████████████| 11.1 MB 3.7 MB/s 
     |████████████████████████████████| 49 kB 2.1 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 124015 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up

In [4]:
import pandas as pd

#Code that solves one instance of the Markdown Pricing Problem

In [2]:
#inputs
list_price = 55.48 #undiscounted price
demand = [120, 160, 194, 223] #for 0%, 15%, 30%, 50% discounts, respectively
weeksleft = 15 #current # of weeks left in the selling season
inv = 1421 #current amount of inventory left
sal = 13.87 #salvage value per unit

In [5]:
#calculate discounted prices
price = [list_price, list_price*(1-.15),list_price*(1-.3), list_price*(1-.5)] #for 0%, 15%, 30%, 50% discounts, respectively

#defining the optimization model
model = ConcreteModel()
model.x = Var(range(4), domain=NonNegativeReals)
model.Objective = Objective(expr = sum(demand[i]*model.x[i]*(price[i]-sal) for i in range(4)) + sal*inv, sense=maximize)
model.InvConstraint = Constraint(expr = sum(demand[i]*model.x[i] for i in range(4)) <= inv)
model.WeeksConstraint = Constraint(expr = sum(model.x[i] for i in range(4)) <= weeksleft)
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {0, 1, 2, 3}

1 Var Declarations
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 4993.2*x[0] + 5326.08*x[1] + 4843.404*x[2] + 3093.0099999999998*x[3] + 19709.27

2 Constraint Declarations
    InvConstraint : Size=1, Index=None, Active=True
        Key  : Lower : Body                                      : Upper  : Active
        None :  -Inf : 120*x[0] + 160*

In [6]:
#solving
opt = SolverFactory('glpk')
opt.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpvjhw3szg.glpk.raw --wglp /tmp/tmpwcundfyt.glpk.glp --cpxlp
 /tmp/tmphuvt6kp0.pyomo.lp
Reading problem data from '/tmp/tmphuvt6kp0.pyomo.lp'...
3 rows, 5 columns, 9 non-zeros
35 lines were read
Writing problem data to '/tmp/tmpwcundfyt.glpk.glp'...
28 lines were written
GLPK Simplex Optimizer, v4.65
3 rows, 5 columns, 9 non-zeros
Preprocessing...
2 rows, 4 columns, 8 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.230e+02  ratio =  2.230e+02
GM: min|aij| =  8.565e-01  max|aij| =  1.168e+00  ratio =  1.363e+00
EQ: min|aij| =  7.336e-01  max|aij| =  1.000e+00  ratio =  1.363e+00
Constructing initial basis...
Size of triangular part is 2
*     0: obj =   1.970927000e+04 inf =   0.000e+00 (4)
*     2: obj =   7.883708000e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40412 bytes)
Writing basic solution to '/tmp/tmpvjhw3szg.glpk.raw'...


{'Problem': [{'Name': 'unknown', 'Lower bound': 78837.08, 'Upper bound': 78837.08, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 5, 'Number of nonzeros': 9, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.013016939163208008}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [7]:
#print results
print('optimal scheduled time at each price level over the remaining weeks: ', (model.x[0](), model.x[1](), model.x[2](), model.x[3]()))
print('projected optimal total revenue: ', model.Objective())

optimal scheduled time at each price level over the remaining weeks:  (11.8416666666667, 0.0, 0.0, 0.0)
projected optimal total revenue:  78837.08000000016


##1. Load the data

In [9]:
df=pd.read_excel('MP_scenarios.xlsx')

In [10]:
df.head()

,No,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value
0,1,55.48,120,160,194,223,15,1421,13.870
1,2,53.68,115,149,171,197,15,2396,13.420
2,3,61.56,140,191,207,459,15,2544,15.390
3,4,65.72,115,151,242,278,15,1316,16.430
4,5,64.98,120,173,221,335,15,1377,16.245


In [13]:
k=0 
rowdata = df.iloc[k].values.tolist()
data = [rowdata[1],
         rowdata[2:6],
         rowdata[6],rowdata[7],rowdata[8]]

##2. Solves for optimal pricing plan and the resulting projected revenue

In [16]:
def solve(data):
  #discounted prices
  price = [data[0], data[0]*(1-.15),data[0]*(1-.3), data[0]*(1-.5)] #for 0%, 15%, 30%, 50% discounts, respectively
  demand=data[1]
  sal=data[4]
  inv=data[3]
  weeksleft=data[2]
  
  #optimization model
  model = ConcreteModel()
  model.x = Var(range(4), domain=NonNegativeReals)
  model.Objective = Objective(expr = sum(demand[i]*model.x[i]*(price[i]-sal) for i in range(4)) + sal*inv, sense=maximize)
  model.InvConstraint = Constraint(expr = sum(demand[i]*model.x[i] for i in range(4)) <= inv)
  model.WeeksConstraint = Constraint(expr = sum(model.x[i] for i in range(4)) <= weeksleft)
  model.pprint()

  #solve
  opt = SolverFactory('glpk')
  opt.solve(model, tee=True)
  objective=model.Objective()
  assignment_matrix = [[model.x[i]() for i in range(4)]]
  return assignment_matrix,objective

In [17]:
#Let's put it in a loop
outputs = []
for k in range(len(df)):
    rowdata = df.iloc[k].values.tolist()
    data = [rowdata[1],
         rowdata[2:6],
         rowdata[6],rowdata[7],rowdata[8]]
    outputs.append(solve(data))

Streaming output truncated to the last 5000 lines.
28 lines were written
GLPK Simplex Optimizer, v4.65
3 rows, 5 columns, 9 non-zeros
Preprocessing...
2 rows, 4 columns, 8 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.750e+02  ratio =  2.750e+02
GM: min|aij| =  7.861e-01  max|aij| =  1.272e+00  ratio =  1.618e+00
EQ: min|aij| =  6.179e-01  max|aij| =  1.000e+00  ratio =  1.618e+00
Constructing initial basis...
Size of triangular part is 2
*     0: obj =   4.405266000e+04 inf =   0.000e+00 (4)
*     3: obj =   1.400234681e+05 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40412 bytes)
Writing basic solution to '/tmp/tmpidc12nc_.glpk.raw'...
17 lines were written
1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {0, 1, 2, 3}

1 Var Declarations
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stal

##3. Output CSV file with added two columns

In [19]:
#add the two columns to data frame
df['Optimized Plan'] = [outputs[i][0] for i in range(len(outputs))]
df['Projected revenue']=[outputs[i][1] for i in range(len(outputs))]

#df to csv
df.to_csv('P2_Krishnaleela.csv', index=False)